In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
desired_directory = '/content/drive/MyDrive'

os.chdir(desired_directory)

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: /content/drive/MyDrive


In [ ]:
cd NLP_AUTUMN_ASSIGNMENT_DATA/NLP_AUTUMN_ASSIGNMENT_DATA

/content/drive/MyDrive/NLP_AUTUMN_ASSIGNMENT_DATA/NLP_AUTUMN_ASSIGNMENT_DATA


In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer, AutoModelForSeq2SeqLM
from collections import Counter


In [ ]:
import numpy as np

In [ ]:

modelname_1="google/flan-t5-small"
modelname_2="google/flan-t5-base"
# Check if CUDA (GPU support) is available
gpu_available = torch.cuda.is_available()

if gpu_available:
    print("GPU is available!")
else:
    print("GPU is not available.")
#Load the tokenizer and model
model_1 = T5ForConditionalGeneration.from_pretrained(modelname_1)#,force_download=True, resume_download=False)
tokenizer_1 = AutoTokenizer.from_pretrained(modelname_1)#,legacy=False,use_fast=False)


#Load the tokenizer and model
model_2 = T5ForConditionalGeneration.from_pretrained(modelname_2)#,force_download=True, resume_download=False)
tokenizer_2 = AutoTokenizer.from_pretrained(modelname_2)#,legacy=False,use_fast=False)



GPU is available!


In [ ]:
#Reading the dataset
path='NLP_ass_test.tsv'
column_names = ['comment','label']

# Load the TSV file into a DataFrame with column names
df = pd.read_csv(path, sep='\t', header=None, names=column_names)

posts= df['comment'].tolist()
Span=df['label'].tolist()
len(posts)


1924

In [ ]:
posts

['i dont think im getting my baby them white 9 he has two white j and nikes not even touched',
 'laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate',
 '<user> what did the old lady do was she a nazi or white supremacist',
 'as much as i appreciate a format of open discussion i really just want a facebook that i can say beaner on',
 'sex be so good a bitch be slow stroking and crying',
 '<user> <user> <user> with a room full of white folks',
 'i do not blame orlando pie rats for loosing like this they are drunk from that black label 😂 😂 😂 mtn 8 ss diski wafa wafa',
 'they are going to figure out a way to kike bitcoin',
 '<user> <user> a camel jockey midde east watering station',
 'then hoes stole my choreo and still managed to make it look bad and lose with it lmfao',
 'the labour party has <number> mps in prison for paedophilia <number> mps and councillors on the sex registry <number> million white 

### zero shot base

In [ ]:
labels=[]
prompt_output=[]
i=0
print("Classify the post delimited by three backticks as either \"normal\" or \"hatespeech\" or \"offensive\".'''{post}'''")
for comment,span in zip(posts,Span):
      #print(span)

      prompt =f"Classify the post delimited by three backticks as either 'normal', 'hatespeech' or 'offensive' and nothing else.'''{comment}'''"
      #Generate the text for the soft hate speech prompt
      inputs = tokenizer_2.encode(prompt, return_tensors="pt")#.to(device)

      outputs = model_2.generate(inputs,temperature=0.01,do_sample = True,max_length = 50)
      #Decode and return the generated counter-speech
      output = tokenizer_2.decode(outputs[0], skip_special_tokens=True)

      prompt_output.append(output)
      i=i+1
      print( output,":",comment,i)



Classify the post delimited by three backticks as either "normal" or "hatespeech" or "offensive".'''{post}'''
normal : i dont think im getting my baby them white 9 he has two white j and nikes not even touched 1
hatespeech : laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate 2
normal : <user> what did the old lady do was she a nazi or white supremacist 3
normal : as much as i appreciate a format of open discussion i really just want a facebook that i can say beaner on 4
offensive : sex be so good a bitch be slow stroking and crying 5
normal : <user> <user> <user> with a room full of white folks 6
normal : i do not blame orlando pie rats for loosing like this they are drunk from that black label 😂 😂 😂 mtn 8 ss diski wafa wafa 7
normal : they are going to figure out a way to kike bitcoin 8
normal : <user> <user> a camel jockey midde east watering station 9
offensive : then hoes stole my choreo and sti

In [ ]:
Counter(prompt_output)

Counter({'normal': 1127, 'hatespeech': 571, 'offensive': 226})

In [ ]:
predicted = prompt_output

In [ ]:
for i in range(len(prompt_output)):
  if prompt_output[i] ==  'neutral' or prompt_output[i] == 'acceptable' or prompt_output[i] == 'usual' or prompt_output[i] == 'common' or prompt_output[i] == 'ordinary' or prompt_output[i] == 'normally' or prompt_output[i] == 'normal':
    predicted.append('normal')
  if prompt_output[i] ==  'insulting' or prompt_output[i] == 'insult' or prompt_output[i] == 'abusive' or prompt_output[i] == 'offense' or prompt_output[i] == "'offensive'" or prompt_output[i] == 'violence' or prompt_output[i] == 'harmful' or prompt_output[i] == 'faggot' or prompt_output[i] == 'offensive' or prompt_output[i] == 'troll' or prompt_output[i] == 'abuse' or prompt_output[i] == "'offensive'":
    predicted.append('offensive')
  if prompt_output[i] ==  'anger' or prompt_output[i] == 'Hatespeech' or prompt_output[i] == 'unusual' or prompt_output[i] == 'unacceptable' or prompt_output[i] == 'hatredspeech' or prompt_output[i] == "'hatespeech'" or prompt_output[i] == 'negative' or prompt_output[i] == 'racist' or prompt_output[i] == 'angerspeech' or prompt_output[i] == 'hatespeech' or prompt_output[i] == 'horrible' or prompt_output[i] == 'a hatespeech' or prompt_output[i] == 'antisocial' or prompt_output[i] == 'hostile' or prompt_output[i] == 'idiocy':
    predicted.append('hatespeech')

In [ ]:
Counter(predicted)

Counter({'normal': 1126, 'hatespeech': 571, 'offensive': 227})

In [ ]:
Counter(Span)

Counter({'normal': 782, 'hatespeech': 594, 'offensive': 548})

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Your list of strings
string_list = ['hatespeech', 'normal', 'offensive']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the list of strings to numerical labels
numerical_labels1 = np.eye(3)[label_encoder.fit_transform(Span)]
numerical_labels2 = np.eye(3)[label_encoder.fit_transform(predicted)]

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the true labels and predicted labels as numpy arrays
true_labels = np.array(numerical_labels1)
predicted_labels = np.array(numerical_labels2)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(true_labels, predicted_labels, average='macro')
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average='macro')
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='macro')
print("F1 score:", f1)

Accuracy: 0.5088357588357588
Precision: 0.48218605634390005
Recall: 0.47514540513650666
F1 score: 0.456100096815892


### Small zero shot

In [ ]:
labels2=[]
prompt_output2=[]
i2=0
print("Classify the post delimited by three backticks as normal or hatespeech or offensive.'''{post}'''")
for comment,span in zip(posts,Span):
      #print(span)

      prompt =f"Classify the post delimited by three backticks as either 'normal', 'hatespeech' or 'offensive' and nothing else.'''{comment}'''"
      #Generate the text for the soft hate speech prompt
      inputs = tokenizer_1.encode(prompt, return_tensors="pt")#.to(device)

      outputs = model_1.generate(inputs,temperature = 0.1, do_sample=True, max_length=50)
      #Decode and return the generated counter-speech
      output = tokenizer_1.decode(outputs[0], skip_special_tokens=True)

      prompt_output2.append(output)
      i2=i2+1
      print( output,":",comment,i2)


Classify the post delimited by three backticks as normal or hatespeech or offensive.'''{post}'''
offensive : i dont think im getting my baby them white 9 he has two white j and nikes not even touched 1
hatespeech : laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate 2
hatespeech : <user> what did the old lady do was she a nazi or white supremacist 3
hatespeech : as much as i appreciate a format of open discussion i really just want a facebook that i can say beaner on 4
offensive : sex be so good a bitch be slow stroking and crying 5
hatespeech : <user> <user> <user> with a room full of white folks 6
offensive : i do not blame orlando pie rats for loosing like this they are drunk from that black label 😂 😂 😂 mtn 8 ss diski wafa wafa 7
offensive : they are going to figure out a way to kike bitcoin 8
offensive : <user> <user> a camel jockey midde east watering station 9
offensive : then hoes stole my cho

In [ ]:
Counter(prompt_output2)

Counter({'offensive': 965,
         'hatespeech': 909,
         'not offensive': 33,
         'normal': 15,
         'hate speech': 1,
         'no': 1})

In [ ]:
predicted = []

In [ ]:
for i in range(len(prompt_output2)):
  if prompt_output2[i] == 'normal' or prompt_output2[i] == 'not' or prompt_output2[i] == 'not offensive' or prompt_output2[i] == 'not children'or prompt_output2[i] == 'no':
    predicted.append('normal')
  if prompt_output2[i] == 'offensive':
    predicted.append('offensive')
  if prompt_output2[i] == 'hate' or prompt_output2[i] == 'hatespeech' or prompt_output2[i] == 'hate speech':
    predicted.append('hatespeech')

In [ ]:
Counter(predicted)

Counter({'offensive': 965, 'hatespeech': 910, 'normal': 49})

In [ ]:
Counter(Span)

Counter({'normal': 782, 'hatespeech': 594, 'offensive': 548})

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Your list of strings
string_list = ['hatespeech', 'normal', 'offensive']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the list of strings to numerical labels
numerical_labels1 = np.eye(3)[label_encoder.fit_transform(Span)]
numerical_labels2 = np.eye(3)[label_encoder.fit_transform(predicted)]

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the true labels and predicted labels as numpy arrays
true_labels = np.array(numerical_labels1)
predicted_labels = np.array(numerical_labels2)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(true_labels, predicted_labels, average='macro')
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average='macro')
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='macro')
print("F1 score:", f1)

Accuracy: 0.29677754677754675
Precision: 0.3509737733275853
Recall: 0.33136093533975314
F1 score: 0.2605737226642666
